In [1]:
import re
import sys
from uuid import uuid1
from time import sleep
from pprint import pprint
from pathlib import Path
from tempfile import gettempdir
from threading import Thread
from functools import partial
from subprocess import Popen, PIPE

from tqdm.notebook import tqdm
import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt

FILENAME = "main.cpp"

In [2]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)

10 variables were found.


,name,method,min,max,log,left,right
0,kErase,suggest_int,1.000,5.0,False,static constexpr auto kErase =,"; // OPTIMIZE [1, 5]"
1,kRadius,suggest_int,2.000,6.0,False,static constexpr auto kRadius =,"; // OPTIMIZE [2, 6]"
2,kAnnealingA,suggest_float,-15.000,15.0,False,static constexpr auto kAnnealingA =,"; // OPTIMIZE [-15.0, 15.0]"
3,kAnnealingB,suggest_float,0.000,3.0,False,static constexpr auto kAnnealingB =,"; // OPTIMIZE [0.0, 3.0]"
4,kAnnealingStart,suggest_float,1.000,100.0,True,static constexpr auto kAnnealingStart =,"; // OPTIMIZE LOG [1.0, 100.0]"
5,kSkipRatio,suggest_float,0.200,0.8,False,static constexpr auto kSkipRatio =,"; // OPTIMIZE [0.2, 0.8]"
6,kTargetDeterminationTrials,suggest_int,1.000,20.0,True,static constexpr auto kTargetDeterminationTria...,"; // OPTIMIZE LOG [1, 20]"
7,kAttractionRatio,suggest_float,0.010,0.9,True,static constexpr auto kAttractionRatio =,"; // OPTIMIZE LOG [0.01, 0.9]"
8,kMaxAttractionDistance,suggest_int,4.000,99.0,True,static constexpr auto kMaxAttractionDistance =,"; // OPTIMIZE LOG [4, 99]"
9,kStartAttraction,suggest_float,0.001,0.9,True,static constexpr auto kStartAttraction =,"; // OPTIMIZE LOG [0.001, 0.9]"


In [3]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]

def read_stream(name, in_file, out_file):
    for line in in_file:
        print(f"[{name}] {line.strip()}", file=out_file)

def run(cmd, name):
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=isinstance(cmd, str))
    stdout_thread = Thread(target=read_stream, args=(name, proc.stdout, sys.stdout))
    stderr_thread = Thread(target=read_stream, args=(name, proc.stderr, sys.stderr))
    stdout_thread.start()
    stderr_thread.start()
    proc.wait()
    return proc

def objective(trial, in_dir, work_dir):
    n_internal_parallel = 3
    
    index_parallel = f"{trial.number:04d}"
    print(f"{index_parallel=}")
    
    work_dir = Path(work_dir)
    directory_input = Path(in_dir)  # 中のすべてのファイルに対して実行される
    #parameters_changed_filename = Path(gettempdir()) / str(uuid1())
    parameters_changed_filename = work_dir / f"{index_parallel}_{FILENAME}"
    
    run(["mkdir", f"{work_dir / index_parallel}_out"], "mkdir")
    run(["mkdir", f"{work_dir / index_parallel}_score"], "mkdir")
    
    # ファイル作成
    run(f"cp {FILENAME} {parameters_changed_filename}", "cp")
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"], log=variable["log"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    #print(command_sed)
    run(command_sed, "sed")
    
    # コンパイル
    command_compile = f"g++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -o {parameters_changed_filename}.out"
    #print(command_compile)
    run(command_compile, "compile")
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        #f"./a.out < ./tools/in/{i:04d}.txt > {out_file} && ./tools/target/release/vis ./tools/in/{i:04d}.txt {out_file}"
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score"
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"cargo run --release --manifest-path ../tools/Cargo.toml --bin tester {directory_input}/@ $(pwd)/{parameters_changed_filename}.out 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        f"./{parameters_changed_filename}.out < {directory_input}/@ > {work_dir / index_parallel}_out/@;"
        f"../tools/target/release/vis {directory_input}/@ {work_dir / index_parallel}_out/@ 2> /dev/null > {work_dir / index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    run(f"find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'", "exec")
    
    # 集計
    scores = []
    for file_path in Path(f"{work_dir / index_parallel}_score/").iterdir():  
        with open(file_path) as f:
            scores.append(int(f.readline().strip().split()[-1]))
    mean_score = sum(scores) / len(scores)
    
    # 後始末
    run(f"rm -rf {work_dir / index_parallel}_out", "rm")
    run(f"rm -rf {work_dir / index_parallel}_score", "rm")
    #run(f"rm {parameters_changed_filename}", "rm")
    run(f"rm {parameters_changed_filename}.out", "rm")
    
    return mean_score

In [4]:
K = 3
N_CLASS = 3

in_dir = Path(f"in_{K}_{N_CLASS}")
work_dir = Path(f"work_{K}_{N_CLASS}")

study_name = f"study_{K}_{N_CLASS}"
storage_path = work_dir / "study.db"
storage = f"sqlite:///{storage_path}"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        index_parallel = f"{trial.number:04d}"
        parameters_changed_filename = work_dir / f"{index_parallel}_{FILENAME}"
        if study.best_value == trial.value:
            print(f"Updated! {study.best_value}")
            !cp {parameters_changed_filename} {work_dir / "best_parameters.cpp"}
        !rm {parameters_changed_filename}
    except:
        print(":(")


[I 2022-08-16 05:03:53,788] Using an existing study with name 'study_3_3' instead of creating a new one.


In [ ]:
study.optimize(
    partial(objective, in_dir=in_dir, work_dir=work_dir),
    n_trials=3000,
    timeout=86400,
    callbacks=[callback]
)

index_parallel='0192'


[I 2022-08-16 08:11:55,522] Trial 192 finished with value: 8164.233766233766 and parameters: {'kErase': 3, 'kRadius': 6, 'kAnnealingA': 13.721431475441573, 'kAnnealingB': 0.9216554043577437, 'kAnnealingStart': 12.515852188787502, 'kSkipRatio': 0.7806734107340336, 'kTargetDeterminationTrials': 7, 'kAttractionRatio': 0.4177089420132005, 'kMaxAttractionDistance': 11, 'kStartAttraction': 0.7933779714338033}. Best is trial 166 with value: 8252.636363636364.


index_parallel='0194'


[I 2022-08-16 08:16:16,968] Trial 194 finished with value: 8130.606060606061 and parameters: {'kErase': 3, 'kRadius': 6, 'kAnnealingA': 12.998239203398484, 'kAnnealingB': 1.2234390145805962, 'kAnnealingStart': 14.713823510587048, 'kSkipRatio': 0.7753508412428399, 'kTargetDeterminationTrials': 8, 'kAttractionRatio': 0.5265392405238577, 'kMaxAttractionDistance': 14, 'kStartAttraction': 0.7054483047938513}. Best is trial 166 with value: 8252.636363636364.


index_parallel='0196'


...

index_parallel='0286'


[I 2022-08-16 11:36:53,238] Trial 286 finished with value: 8183.8138528138525 and parameters: {'kErase': 3, 'kRadius': 6, 'kAnnealingA': 11.135797620197927, 'kAnnealingB': 0.6901901843069648, 'kAnnealingStart': 14.529378084769805, 'kSkipRatio': 0.77883314647686, 'kTargetDeterminationTrials': 9, 'kAttractionRatio': 0.43980497547736236, 'kMaxAttractionDistance': 9, 'kStartAttraction': 0.7306023957317709}. Best is trial 166 with value: 8252.636363636364.


index_parallel='0288'


[I 2022-08-16 11:41:15,155] Trial 288 finished with value: 8159.792207792208 and parameters: {'kErase': 3, 'kRadius': 6, 'kAnnealingA': 9.125520742232712, 'kAnnealingB': 0.4494968187727376, 'kAnnealingStart': 15.210193796291529, 'kSkipRatio': 0.789779498280043, 'kTargetDeterminationTrials': 13, 'kAttractionRatio': 0.38648664116304365, 'kMaxAttractionDistance': 8, 'kStartAttraction': 0.8973448794943635}. Best is trial 166 with value: 8252.636363636364.


index_parallel='0290'
